# Brincando com dados: Ganhadores do Oscar Parte 1

Olá a todos, esse post será o primeiro de muitos (como diria a minhoca do worms) sobre análise de dados. O objetivo desses posts intitulados “Brincando com os dados” é dar exemplos de como importar, limpar, analisar dados e muito mais.

O assunto desse primeiro exemplo vai ser “Ganhadores do Oscar” e o objetivo dele é ser bem simples e didático, mostrar alguns conceitos e exemplos de ferramentas e bibliotecas. 

A primeira parte será sobre como importar a base de dados. Irei mostrar como utilizar a biblioteca [IMDbPY](http://imdbpy.sourceforge.net) para buscar filmes e o [PyMongo](https://api.mongodb.org/python/current/) para salvar os dados no [MongoDB](https://www.mongodb.org/).

De forma que o post fique mais simples em termos de código, estou usando aqui o html gerado pelo próprio IPython Notebook, então os comentários e o código estão entrelaçado.

## Preparativos
Para ficar mais simples e didático decidimos utilizar o IPython Notebook, para fazer a instalação dele siga o tutorial descrito aqui: http://jupyter.readthedocs.org/en/latest/install.html 

Para rodar o código ou acompanhar o post pelo IPython Notebook baixe o arquivo: ARQUIVO.PY

## Base
MongoDB: O MongoDB é um banco de dados não relacional, ele é orientado a documentos e utiliza um formato chamado BSON, bem próximo ao JSON. Ele é bem simples de entender e utilizar.

IMDbPY: IMDbPy é uma biblioteca em python para acessar os dados do IMDb.

PyMongo: PyMongo é uma biblioteca em python para acessar o MongoDB.


## Let’s do this! (Leeroy Jenkins)


In [2]:
from imdb import IMDb
import pymongo
from pymongo import MongoClient   

In [3]:
movies = {}
movies['2017'] = ['Arrival','Fences', 'Hacksaw Ridge', 'Hell or High Water', 'Hidden Figures', 'La la land', 'Lion', 'Manchester by the sea', 'Moonlight']
movies['2016'] = ['The Big Short','Bridge of Spies','Brooklyn','Mad Max: Fury Road','The Martian','The Revenant','Room','Spotlight']
movies['2015'] = ['Birdman','American Sniper','Boyhood','The Grand Budapest Hotel','Imitation Game','Selma','The Theory of Everything','Whiplash']
movies['2014'] = ['American Hustle','Captain Phillips','Dallas Buyers Club','Gravity','Her (2013)','Nebraska','Philomena','The Wolf of Wall Street', '12 Years a Slave']
movies['2013'] = ['Argo','Amour','tt2125435','Django Unchained','tt1707386','Life of Pi','Lincoln','Silver Linings Playbook','Zero Dark Thirty']
movies['2012'] = ['The Artist','The Descendants','Extremely Loud & Incredibly Close','The Help','Hugo','Midnight in Paris','Moneyball','The Tree of Life','War Horse']
movies['2011'] = ['The King\'s Speech','127 Hours','Black Swan','The Fighter','Inception','The Kids Are All Right','The Social Network','Toy Story 3','True Grit','Winter\'s Bone']
movies['2010'] = ['The Hurt Locker','Avatar','tt0878804','District 9','An Education (2009)','Inglorious Bastards','Precious','A Serious Man','up','Up in the air (2009)']
movies['2009'] = ['Slumdog Millionaire','Benjamin Button','tt0870111','Milk','The Reader']
movies['2008'] = ['No Country for Old Men','Atonement','Juno','Michael Clayton','There Will Be Blood']
movies['2007'] = ['The Departed','Babel','Letters from Iwo Jima','Little Miss Sunshine (2006)','The Queen (2006)']
movies['2006'] = ['Crash','Brokeback Mountain','Capote','Good Night, and Good Luck.','tt0408306']
movies['2005'] = ['Million Dollar Baby','The Aviator','Finding Neverland','Ray','Sideways']
movies['2004'] = ['The Lord of the Rings: The Return of the King','Lost in Translation','Master and Commander: The Far Side of the World','Mystic River','Seabiscuit']
movies['2003'] = ['Chicago','tt0217505','tt0274558','The Lord of the Rings: The Two Towers','The Pianist']
movies['2002'] = ['A Beautiful Mind','Gosford Park','tt0247425','The Lord of the Rings: The Fellowship of the Ring','Moulin Rouge!']      
movies['2001'] = ['Gladiator','Chocolat','Crouching Tiger, Hidden Dragon','Erin Brockovich','Traffic']          

exceptions = {'tt0878804':'The Blind Side','tt1707386':'Les Misérables', 'tt0870111':'Frost/Nixo', 'tt2125435': 'Beasts of the Southern Wild', 'tt0247425': 'In the Bedroom', 'tt0408306':'Munich', 'tt0274558':'The Hours', 'tt0217505':'Gangs of New York'}



Primeiramente, foi feito o import das bibliotecas que vamos precisar. Logo depois, criamos a lista dos filmes indicados ao Oscar de “Melhor Filme” de 2000 a 2014. 

É importante salientar, que a melhor forma de utilizar o IMDbPy não é essa. O processo normal dessa biblioteca é importar a base inteira do IMDb para seu MySQL [tutorial](http://imdbpy.sourceforge.net/docs/README.sqldb.txt) , mas como queria demonstrar o MongoDB, decidi fazer dessa maneira.

### First, we are going to insert the Oscar Nominees in the MongoDB database


In [4]:
# initiating the imdb api
ia = IMDb()

In [5]:
# starting the client
client = MongoClient()

In [6]:
# getting the db for the imdb database
db = client.imdb

In [7]:
# getting the movies colletion
movie_db = db.movies

Agora, iniciamos a api do IMDb e o cliente do MongoDB, logo estamos definindo que iremos trabalhar na database chamada ‘imdb’ e a variável movie_db será utilizada para trabalha na coleção ‘movies’. 

In [25]:
# creating a function that maps the movie json to a dictionary
def map_movie_to_dic(title, movie, year, exceptions):
              
    movie_dic = {}
    movie_dic['_id'] = movie.movieID
    movie_dic['oscar_year'] = year
    map_person_list(movie,movie_dic,'assistant director')
    map_person_list(movie,movie_dic,'director')   
    map_person_list(movie,movie_dic,'writer')  
    map_person_list(movie,movie_dic,'editor')
    map_person_list(movie,movie_dic,'cast') 
    map_person_list(movie,movie_dic,'editor') 
    map_person_list(movie,movie_dic,'original music')
    map_company_list(movie,movie_dic,'distributors')
    map_company_list(movie,movie_dic,'production companies')  
    map_company_list(movie,movie_dic,'special effects companies')         
    map_person_list(movie,movie_dic,'casting director') 
    
    if title in exceptions.keys():
        movie_dic['title'] = exceptions[title]
    else:
        movie_dic['title'] = title
    
    print 'Title searched: ' + title + ' - Title in the result: ' + movie['title']
        
    if 'plot' in movie.data:
        movie_dic['plot'] = movie['plot']
    if 'rating' in movie.data:    
        movie_dic['user_rating'] = movie['rating']
    else:
        print 'Movie ' + title + ' has no rating\n' + str(movie.data)        
    if 'countries' in movie.data:    
        movie_dic['countries'] = movie['countries']
    if 'genres'  in movie.data:
        movie_dic['genres'] = movie['genres']      
    else:
        print 'Movie ' + title + ' has no genres\n' + str(movie.data)
    if 'runtime'  in movie.data:
        movie_dic['runtime'] = movie['runtime']
    if 'languages' in movie.data:
        movie_dic['languages'] = movie['languages']  
    if 'opening weekend' in movie['business']:
        movie_dic['opening_weekend'] = get_money(movie['business']['opening weekend'])
    else:
        movie_dic['opening_weekend'] = 0  
    if 'budget' in movie['business']:
        movie_dic['budget'] = get_money(movie['business']['budget'])
    else:
        movie_dic['budget'] = 0        

    return movie_dic

A função *map_movie_to_dic()* mapeia o resultado da biblioteca IMDbPy (em formato JSON) para um dicionário de python. Mas para isso são necessárias as funções listadas abaixo.

In [26]:
import re
# Creating a function that maps the money figure to a integer
def get_money(raw):
    regex = '\$([0-9],*)*'
    for item in raw:
        match = re.match(regex, item)
        if match:
            # return the first group matched without the $ and commas
            return int(match.group(0).strip('$').replace(',',''))

A função *get_money()* utiliza de um regex simples para transformar uma string no formato $100,00 em um inteiro 100.

In [27]:
# Creating function that maps a person list. Saving only the name and id
def map_person_list(movie,movie_dic,field):
    movie_dic[field] = []  
    if field in movie.data:
        for person in movie[field]:    
            subdoc = {}
            subdoc['name'] = person['name']
            subdoc['id'] = person.personID    
            movie_dic[field].append(subdoc)

A função *map_person_list()* cria uma lista com dicionários, a idéia é que em um filme pode existir mais de um diretor, escritor e ator. 

In [28]:
# Creating function that maps a company list. Saving only the name and id
def map_company_list(movie,movie_dic,field):
    movie_dic[field] = []  
    if field in movie.data:
        for person in movie[field]:    
            subdoc = {}
            subdoc['name'] = person['name']
            subdoc['id'] = person.companyID    
            movie_dic[field].append(subdoc)

A função *map_company_list* é muito parecida com a *map_person_list()* mas é utilizada para campos relativos a empresas.

In [29]:
import time

# iterating in the movies list by year
for year in movies:
    
    #for each movie
    for movie in movies[year]:        
        # setting the number of retries
        for attempt in range(15):        
            # Check if this movie is already in the MongoDB database
            title = ''
            if movie in exceptions.keys():
                title = exceptions[movie]
            else:
                title = movie
            
            if movie_db.find_one({'title':title}):
                print 'Movie '+ movie +' already in mongodb'
                break;

            # search in the IMDB database by title
            try:
                imdb_result = ia.search_movie(movie)
            except Exception as e:
                print 'Oops!  Error getting movie \n' + str(e) 
                time.sleep(10)

            # if the result is not empty
            if len(imdb_result) > 0:
                best_match = imdb_result[0]

                #update necessary fiels
                ia.update(best_match)
                ia.update(best_match, 'business')

                # saving the movie in the database. 
                # pymongo is easy, just save the dictionary
                # Observation: if the database is going to be big, it is better to use
                # short field names in MongoDB, example 'title' turns into 't'
                movie_db.save(map_movie_to_dic(movie, best_match, int(year), exceptions))
                print 'Just saved the movie ' + movie  +' into the database'
                
                # sucess will break the attempts while
                break

            else:
                print 'No result for the movie: ' + movie +'. Trying again after 30 seconds.'
                ia = IMDb()
                time.sleep(30)

Movie Slumdog Millionaire already in mongodb
Movie Benjamin Button already in mongodb
Movie tt0870111 already in mongodb
Movie Milk already in mongodb
Movie The Reader already in mongodb
Movie Chicago already in mongodb
Movie tt0217505 already in mongodb
Movie tt0274558 already in mongodb
Title searched: The Lord of the Rings: The Two Towers - Title in the result: The Lord of the Rings: The Two Towers
Just saved the movie The Lord of the Rings: The Two Towers into the database


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead


Title searched: The Pianist - Title in the result: The Pianist
Just saved the movie The Pianist into the database
Title searched: A Beautiful Mind - Title in the result: A Beautiful Mind
Just saved the movie A Beautiful Mind into the database
Title searched: Gosford Park - Title in the result: Gosford Park
Just saved the movie Gosford Park into the database
Title searched: tt0247425 - Title in the result: In the Bedroom
Just saved the movie tt0247425 into the database
Title searched: The Lord of the Rings: The Fellowship of the Ring - Title in the result: The Lord of the Rings: The Fellowship of the Ring
Just saved the movie The Lord of the Rings: The Fellowship of the Ring into the database
Title searched: Moulin Rouge! - Title in the result: Moulin Rouge!
Just saved the movie Moulin Rouge! into the database
Title searched: No Country for Old Men - Title in the result: No Country for Old Men
Just saved the movie No Country for Old Men into the database
Title searched: Atonement - Titl

## Importando os filmes
O código acima itera nos anos selecionados, para cada ano ele pega a lista de filmes. 

Cada filme é requisitado no banco de dados pelo seu título( *movie_db.find_one({'title':title})* ), se não for encontrado então iremos na api do IMDb (usando a função *search_movie* ).

Como a api não é muito confiável/estável coloquei um *for* com 15 tentativas por filme. 

A função *ia.update()* é utilizada para buscar mais campos de cada filme.

A api retorna um JSON representando o filme, utilizamos então a função *map_movie_to_dic()* para transformar ela em um dicionário de python. Esse dicionário pode ser salvo diretamente na coleção 'movies' utilizando a função *save* do pyMongo(sim, é simples assim).


## Fazendo atualização dos ganhadores do Oscar

In [38]:
# first we are going to create a list with all oscar winners.
# I could not find this information in the IMDB api
oscar_winners = ['Gladiator', 'A Beautiful Mind','Chicago', 'The Lord of the Rings: The Return of the King', 
                 'Million Dollar Baby', 'Crash', 'The Departed', 'No Country for Old Men', 'Slumdog Millionaire',
                 'The Hurt Locker', 'The King\'s Speech','The Artist', 'Argo', '12 Years a Slave', 'Birdman', 'Spotlight']


Primeiramente, listaremos aqui todos ganhadores do Oscar de melhor filme de 2000 a 2016

In [40]:
# Now we are going to iterate trough this list updating a new field
# with mongodb you do not have a schema, so we can update a non-existent field. It will create the field.

db.movies.update_many({},{"$set": { "winner": False}})
    
for winner in oscar_winners:
    # the update with pymongo is simple, is json-like operation
    # .update_one({QUERY}, {UPDATE})  https://docs.mongodb.org/getting-started/python/update/
    db.movies.update_one({'title': winner},{"$set": { "winner": True}})

# Now we can check if everything went smoothly, lets query for the oscar winners
# we are sorting by the oscar year (descending)
cursor = movie_db.find({"winner": True}).sort('oscar_year', pymongo.DESCENDING)
print 'The list to be updated has ' + str(len(oscar_winners)) + ' movies, the query returned '+ str(cursor.count())
for document in cursor:
    print(document['title'])

The list to be updated has 16 movies, the query returned 16
Spotlight
Birdman
12 Years a Slave
Argo
The Artist
The King's Speech
The Hurt Locker
Slumdog Millionaire
No Country for Old Men
The Departed
Crash
Million Dollar Baby
The Lord of the Rings: The Return of the King
Chicago
A Beautiful Mind
Gladiator


Agora iteramos pela lista de ganhadores e realizamos um update no documento adicionando um novo campo chamado "winner", esse campo será *True* se esse filme ganhou o oscar e não existirá se ele não ganhou.

O update no pymongo é simples: o primeiro parâmetro é a consulta que será realizada, no nosso caso, queremos o filme que o campo 'title' seja igual ao título do filme que ganhou o oscar. O segundo parâmetro é a operação de atualização que será realizada nos documentos que foram buscados pela consulta, nesse caso, estamos usando o operador de *$set* para atualizar/criar um campo chamado 'winner' e colocando o valor True para ele.

Depois disso, fazemos uma consulta por todos os filmes que tem o campo *winner* igual a True ordenando por data(invertido). Conferimos então o total da lista buscada pelo *find* com a lista inicial.

In [37]:
# Now lets list all movies from 2013 and then that oscar winner from that year
cursor = movie_db.find({'oscar_year':2016})
print('The nominees for best picture in 2016 were:\n')
for document in cursor:
    print(document['title'])
    
result = movie_db.find_one({'oscar_year':2016,'winner':True})    
print('\nThe Winner from 2016 was: \n'+ result['title'])

The nominees for best picture in 2016 were:

The Big Short
Bridge of Spies
Brooklyn
Mad Max: Fury Road
The Martian
The Revenant
Room
Spotlight

The Winner from 2016 was: 
Spotlight


Por último, vamos listar todos os filmes de 2016 seguido pelo ganhador do mesmo.


Finalizando esse código temos um banco de dados no Mongodb chamado **"imdb"** com uma coleção **"movies"** com todos os filmes indicados ao Oscar de melhor de 2000 a 2016.

No próximo post iremos utilizar esses filmes para analisar os dados e tentar descobrir peculiaridades e padrões entre os filmes ganhadores.

Abraços e até o próximo

## Remover esse codigo antes de enviar

In [14]:

cursor = movie_db.find({"rotten_critic":{"$exists":False}})
print('Add the Rotten Tomatoes score for each movie.\n')
for document in cursor:
    print('\n')
    print(document['title'])
    critic = raw_input('Score for critics: ')
    people = raw_input('Score for people: ')
    
    db.movies.update_one({'title': document['title']},{"$set": { "rotten_critic": critic, "rotten_people": people}})    

Add the Rotten Tomatoes score for each movie.



Amour
Score for critics: 93
Score for people: 82


Beasts of the Southern Wild
Score for critics: 86
Score for people: 76


Django Unchained
Score for critics: 88
Score for people: 91


Les Misérables
Score for critics: 69
Score for people: 79


Life of Pi
Score for critics: 87
Score for people: 94


Lincoln
Score for critics: 90
Score for people: 80


Silver Linings Playbook
Score for critics: 92
Score for people: 86


Zero Dark Thirty
Score for critics: 92
Score for people: 80


The Artist
Score for critics: 96
Score for people: 87


The Descendants
Score for critics: 89
Score for people: 79


Extremely Loud & Incredibly Close
Score for critics: 46
Score for people: 61


The Help
Score for critics: 75
Score for people: 89


Hugo
Score for critics: 94
Score for people: 78


Midnight in Paris
Score for critics: 93
Score for people: 83


Moneyball
Score for critics: 94
Score for people: 86


The Tree of Life
Score for critics: 84
Score fo